In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import os

In [18]:
#Load images function
def load_image_from_folder(folder, label, target_size=(256, 256)):
    """
    Loads images from a specified folder, resizes them to a target size,
    and assigns a label to each image.
    Args:
        folder (str): The path to the folder containing the images.
        label (str): The label to assign to each image.
        target_size (tuple): The desired size to resize the images.
    Returns:
        tuple: A tuple containing two lists: the list of loaded images and the list of corresponding labels.
    """
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
            image_path = os.path.join(folder, filename)
            image_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image_gray is not None:
                image_resized = cv2.resize(image_gray, target_size)
                images.append(image_resized)
                labels.append(label)
    return images, labels



In [31]:
#Load images
Normal_folder ='train/NORMAL/'
Pneumonia_folder = 'train/PNEUMONIA/'

normal_images, normal_labels = load_image_from_folder(Normal_folder, 0, target_size=(256, 256))
pneumonia_images, pneumonia_labels = load_image_from_folder(Pneumonia_folder, 1, target_size=(256, 256))


In [32]:
#Prepare data
images = np.array(normal_images + pneumonia_images, dtype=np.uint8)
labels = np.array(normal_labels + pneumonia_labels, dtype=np.uint8)

#Verify if the images are loaded correctly
if len(images) == len(labels):
    print("Images loaded correctly")
else:
    print("Images not loaded correctly")


Images loaded correctly


In [33]:
#Normalize the images
images = images / 255.

#Divide data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
print(X_train.shape)

(4172, 256, 256)


In [34]:
from keras import layers, models
#model architecture
model = keras.Sequential()
model.add(layers.Flatten(input_shape=(256, 256)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [35]:
#Compile the model
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

#train the model
model.fit(X_train, y_train, epochs=10)


Epoch 1/10
131/131 [==============================] - 63s 472ms/step - loss: 1.9694 - accuracy: 0.8061
Epoch 2/10
131/131 [==============================] - 59s 449ms/step - loss: 0.3439 - accuracy: 0.9056
Epoch 3/10
131/131 [==============================] - 60s 457ms/step - loss: 0.1717 - accuracy: 0.9382
Epoch 4/10
131/131 [==============================] - 76s 579ms/step - loss: 0.1890 - accuracy: 0.9305
Epoch 5/10
131/131 [==============================] - 66s 503ms/step - loss: 0.2154 - accuracy: 0.9235
Epoch 6/10
131/131 [==============================] - 65s 499ms/step - loss: 0.1375 - accuracy: 0.9487
Epoch 7/10
131/131 [==============================] - 71s 542ms/step - loss: 0.1560 - accuracy: 0.9432
Epoch 8/10
131/131 [==============================] - 70s 532ms/step - loss: 0.1445 - accuracy: 0.9446
Epoch 9/10
131/131 [==============================] - 65s 499ms/step - loss: 0.1383 - accuracy: 0.9449
Epoch 10/10
131/131 [==============================] - 68s 520ms/step - l

In [36]:
#Save the model
model.save('model_pneumonia.h5')
print("Model saved")

Model saved


In [37]:
#Test the model
model.evaluate(X_test, y_test)



33/33 [==============================] - 2s 54ms/step - loss: 0.1917 - accuracy: 0.9224


[0.1917264759540558, 0.9224137663841248]